In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17778, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 17778 (delta 97), reused 50 (delta 50), pack-reused 17626 (from 4)
Receiving objects: 100% (17778/17778), 17.09 MiB | 20.59 MiB/s, done.
Resolving deltas: 100% (12074/12074), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.1 MB/s eta 0:00:00


In [ ]:
!pip install opencv-python


In [ ]:
video_path = '/content/drive/MyDrive/ADAS_Final_Akshada/video/your_video.mp4'


In [ ]:
import cv2
import numpy as np
import torch
from IPython.display import clear_output


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.classes = [2, 3, 5, 7]   # car, motorcycle, bus, truck


/usr/local/lib/python3.12/dist-packages/torch/hub.py:335: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2026-1-10 Python-3.12.12 torch-2.9.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 188MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
def region_of_interest(img):
    h, w = img.shape[:2]
    polygon = np.array([
        [(0, h), (w, h), (w, int(h*0.55)), (0, int(h*0.55))]
    ])

    mask = np.zeros_like(img)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(img, mask)

def detect_lanes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    cropped = region_of_interest(edges)

    lines = cv2.HoughLinesP(
        cropped,
        rho=1,
        theta=np.pi/180,
        threshold=50,
        minLineLength=40,
        maxLineGap=50
    )

    lane_frame = frame.copy()
    left_lines, right_lines = [], []
    h, w = frame.shape[:2]
    mid = w//2

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2-y1)/(x2-x1+1e-6)
            if abs(slope) < 0.5:
                continue
            if slope < 0 and x1 < mid and x2 < mid:
                left_lines.append(line[0])
            elif slope > 0 and x1 > mid and x2 > mid:
                right_lines.append(line[0])

    def avg_line(lines):
        if len(lines) == 0:
            return None
        x1 = np.mean([l[0] for l in lines]).astype(int)
        y1 = np.mean([l[1] for l in lines]).astype(int)
        x2 = np.mean([l[2] for l in lines]).astype(int)
        y2 = np.mean([l[3] for l in lines]).astype(int)
        return x1, y1, x2, y2

    left = avg_line(left_lines)
    right = avg_line(right_lines)

    if left is not None:
        cv2.line(lane_frame, (left[0],left[1]), (left[2],left[3]), (0,255,0), 6)
    if right is not None:
        cv2.line(lane_frame, (right[0],right[1]), (right[2],right[3]), (0,255,0), 6)

    return lane_frame, left, right


In [ ]:
ROI_POLYGON = np.array(
    [[(100,480), (540,480), (360,280), (120,280)]]
)

def in_roi(xc, yc):
    return cv2.pointPolygonTest(ROI_POLYGON, (xc, yc), False) >= 0

counted_ids = set()

def get_centroid(x1,y1,x2,y2):
    return (int((x1+x2)/2), int((y1+y2)/2))



In [ ]:
def collision_warning(box, frame_h):
    x1,y1,x2,y2 = box
    height = y2 - y1

    if height > frame_h * 0.35:       # bounding box too large
        return True

    if y2 > frame_h * 0.80:           # very close to bottom
        return True

    return False


In [ ]:
video_path = "/content/drive/MyDrive/ADAS_Final_Akshada/video/your_video.mp4"
output_path = "/content/drive/MyDrive/ADAS_Final_Akshada/outputs/final_output.mp4"


In [ ]:
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (640,480))

total_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640,480))
    h, w = frame.shape[:2]

    # ---- Lane Detection ----
    lane_frame, left, right = detect_lanes(frame)

    # ---- YOLO Detection ----
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()

    for det in detections:
        x1,y1,x2,y2,conf,cls = det
        x1,y1,x2,y2 = map(int, (x1,y1,x2,y2))

        cx, cy = get_centroid(x1,y1,x2,y2)

        # Count when entering ROI
        if in_roi(cx, cy):
            if id(det) not in counted_ids:
                counted_ids.add(id(det))
                total_count += 1

        # Draw vehicle box
        cv2.rectangle(lane_frame, (x1,y1), (x2,y2), (255,0,0), 2)
        cv2.circle(lane_frame, (cx,cy), 4, (0,0,255), -1)

        # Collision Warning
        if collision_warning((x1,y1,x2,y2), h):
            cv2.putText(lane_frame, "WARNING: COLLISION RISK!", (10,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 3)

    # ---- Lane Departure Warning ----
    if left and right:
        lane_center = (left[0] + right[0]) // 2
        deviation = abs(lane_center - w//2)
        if deviation > 80:
            cv2.putText(lane_frame, "LANE DEPARTURE!", (10,90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 3)

    # ---- ROI Mask ----
    cv2.polylines(lane_frame, ROI_POLYGON, True, (0,255,255), 3)

    # ---- Vehicle Count Display ----
    cv2.putText(lane_frame, f"Vehicles: {total_count}", (10,450),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    out.write(lane_frame)

cap.release()
out.release()
print("Processing complete! Output saved at:", output_path)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.am

Processing complete! Output saved at: /content/drive/MyDrive/ADAS_Final_Akshada/outputs/final_output.mp4
